In [ ]:
%pip install tensorflow opencv-python matplotlib

In [ ]:
# Standard dependencies
import cv2
import os
import random
import numpy as np # For re-shaping arrays
from matplotlib import pyplot as plt # Visualise images

# Tensorflow dependencies
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [ ]:
# To avoid running out of memory, we restrict the GPU memory growth aka
# how many resources the model is consuming at any given time
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    # Just in case the device we are using has more than one gpu, we are 
    # making sure to restrict the usage of ALL of them
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Create files that will hold the anchor, positive and negative images:
# Anchor: The image we imput
# Positive: Images that match the anchor
# Negative: Images that are different from the anchor
anc_path = os.path.join("data","anchor")
pos_path = os.path.join("data","positive")
neg_path = os.path.join("data","negative")

os.makedirs(anc_path)
os.makedirs(pos_path)
os.makedirs(neg_path)


In [ ]:
# Uncompress the tar file that contains the images in the Wild Dataface
!tar -xf lfw.tgz

In [ ]:
# We take the images that we downloaded and place in the negative folder
# (these images will be used so that the machine can understand that the
# person whose image we are providing isn't the same as any of the ones in
# the negative folder)

# Go through all the directories in the lfw folder
for directory in os.listdir('lfw'):
    # Find all the images in said directory
    for file in os.listdir(os.path.join('lfw',directory)):
        # Replace the path of that image with the path of the negative folder
        # (aka place the image in the negative folder)
        previous_path = os.path.join('lfw', directory, file)
        new_path = os.path.join(neg_path, file)
        os.replace(previous_path, new_path)

In [ ]:
# Importing this so that the images we save  all have different names
import uuid

# Now we will get the images we will require for the anchor and positive files

# Connect to the webcam
capture = cv2.VideoCapture(0) # Keep in mind this number might vary slightly
                              # so try out a few other numbers like 1, 2, 3, 4, 5 etc in case there is a problem
while (capture.isOpened()):
    return_value, frame = capture.read()

    # Keeping in mind that the images in the negative folder have a resolution of 250x250
    # we need out frames (aka the images we will capture) to be 250x250 as well
    frame = frame[120:370, 200:450, :]

    # Show the camera feed
    cv2.imshow("Images", frame)

    # Add image to anchor if 'a' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('a')):
        # Create the unique name and save the image
        name = os.path.join(anc_path, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)
    
    # Add image to positive if 'p' is pressed
    if (cv2.waitKey(1) & 0XFF == ord('p')):
        # Create the unique name and save the image
        name = os.path.join(pos_path, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(name, frame)

    # Break by pressing he 'q' key
    if (cv2.waitKey(1) & 0XFF == ord('q')):
        break

# Release webcam
capture.release()
# Close the camera feed window
cv2.destroyAllWindows()

In [ ]:
# Get 300 image paths from each image set
anchor = tf.data.Dataset.list_files(anc_path+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(pos_path+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(neg_path+'\*.jpg').take(300)

In [ ]:
# Scale and resize the images
def preprocess(file_path):
    # Get byte code of image (the file path) and then decode it
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)

    img = tf.image.resize(img, (105, 105)) # Resizing out image according to the "Siamese Neural Networks"
                                           # research paper
    img = img / 255 # Scale every pixel value to 0-1 => scale the image
    return img

In [ ]:
# Depending on the inputs (anchor, positive) or (anchor, negative) we will be
# getting a result ( a label ) as follows:
# (anchor, positive) => 1
# (anchor, negative) => 0

positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives) # Combine the positives and negatives

In [ ]:
samples = data.as_numpy_iterator()

In [ ]:
example = samples.next()


In [ ]:
# Create function to scale and resize both images we pass
def twin_preprocess(anc, verification_image, label):
    return (preprocess(anc), preprocess(verification_image), label)

res = twin_preprocess(*example)
plt.imshow(res[0])

In [ ]:
# Dataloader pipeline
data = data.map(twin_preprocess) # ERROR TO_DO
data = data.cache()
data = data.shuffle(buffer_size=1024) # Simply mix the positive and negative images

In [ ]:
# Training partition
train_data = data.take((round(len(data)*0.7))) # Get 70% of the samples
train_data = train_data.batch(16) # Pass 16 images each time
train_data = train_data.prefetch(8) # Preprocess the next image beforehand


In [ ]:
train_sumples = train_data.as_numpy_iterator()

len(train_sumples.next()[0])

In [ ]:
# Testing Partition
test_data = data.skip((round(len(data)*0.7)))
test_data = test_data.take((round(len(data)*0.3)))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)